In [4]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
import matplotlib.pyplot as plt
import os
import cv2
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
import io
import imageio
import ipywidgets
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from keras import backend as K
import mediapipe as mp

ModuleNotFoundError: ignored

## Preprocess Videos with MediaPipe and Black Background



In [ ]:
import os
import cv2
import numpy as np
import mediapipe as mp
max_videos_per_class = 30  # Maximum videos to process per class
min_frames_required = 30  # Minimum frames required in each video
videos_to_skip = 3  #
mp_holistic = mp.solutions.holistic.Holistic(static_image_mode=False, min_detection_confidence=0.2, min_tracking_confidence=0.2)
mp_hands = mp.solutions.hands.Hands(static_image_mode=False, min_detection_confidence=0.2, min_tracking_confidence=0.2)

def apply_mediapipe_holistic(frame):
    # Create a black background
    global mp_holistic
    black_frame = frame.copy()
    black_frame[:] = (0, 0, 0)

    # Convert the frame to RGB (Mediapipe requires RGB input)
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # Process the frame with Mediapipe Holistic
    results = mp_holistic.process(frame_rgb)

    # Draw the pose landmarks with orange color
    if results.pose_landmarks:
        mp.solutions.drawing_utils.draw_landmarks(
            black_frame, results.pose_landmarks, mp.solutions.holistic.POSE_CONNECTIONS,
            landmark_drawing_spec=mp.solutions.drawing_utils.DrawingSpec(color=(255, 165, 0), thickness=1, circle_radius=1),
            connection_drawing_spec=mp.solutions.drawing_utils.DrawingSpec(color=(255, 165, 0), thickness=1))

    # Draw the left hand landmarks with blue color
    if results.left_hand_landmarks:
        mp.solutions.drawing_utils.draw_landmarks(
            black_frame, results.left_hand_landmarks, mp.solutions.holistic.HAND_CONNECTIONS,
            landmark_drawing_spec=mp.solutions.drawing_utils.DrawingSpec(color=(0, 0, 255), thickness=1, circle_radius=1),
            connection_drawing_spec=mp.solutions.drawing_utils.DrawingSpec(color=(0, 0, 255), thickness=1))

    # Draw the right hand landmarks with purple color
    if results.right_hand_landmarks:
        mp.solutions.drawing_utils.draw_landmarks(
            black_frame, results.right_hand_landmarks, mp.solutions.holistic.HAND_CONNECTIONS,
            landmark_drawing_spec=mp.solutions.drawing_utils.DrawingSpec(color=(128, 0, 128), thickness=1, circle_radius=1),
            connection_drawing_spec=mp.solutions.drawing_utils.DrawingSpec(color=(128, 0, 128), thickness=1))

    return black_frame

# Define the path to the main dataset folder
main_dataset_path = '/content/drive/MyDrive/vivit/dataset/Videos'  # original video dataset foldedr
output_folder = '/content/drive/MyDrive/vivit/MediaPipeProcessed/Videos'  # Output folder 

# Make sure the output folder exists or create it
os.makedirs(output_folder, exist_ok=True)

class_folders = os.listdir(main_dataset_path)
for class_index, class_folder in enumerate(class_folders):
    print(class_folder)
    class_folder_path = os.path.join(main_dataset_path, class_folder)

    # Track the number of videos processed for each class
    videos_processed = 0

    skipped_videos_per_class = []  # List to store skipped video names for each class

    for filename in os.listdir(class_folder_path):
        if videos_processed >= max_videos_per_class:
            break  # Stop if we have reached the limit for this class

        if videos_processed < videos_to_skip:
            # Skip the first 'videos_to_skip' videos in each class
            videos_processed += 1
            skipped_videos_per_class.append(filename)
            continue

        video_file_path = os.path.join(class_folder_path, filename)

        cap = cv2.VideoCapture(video_file_path)
        video_frames = []

        frame_count = 0  # Count frames in the video

        while cap.isOpened():
            ret, frame = cap.read()
            if not ret:
                break
            video_frames.append(frame)
            frame_count += 1

        cap.release()

        if frame_count >= min_frames_required:
            video = np.array(video_frames)

            # Create black background frames for each video
            black_frames = np.zeros_like(video)

            # Apply MediaPipe Holistic to each frame and overlay keypoints
            for i in range(video.shape[0]):
                black_frame = apply_mediapipe_holistic(video[i])
                black_frames[i] = black_frame

            # Save processed videos with keypoints for each class in MP4 format
            class_output_folder = os.path.join(output_folder, class_folder)
            os.makedirs(class_output_folder, exist_ok=True)

            output_video_path = os.path.join(class_output_folder, f'{class_index}_{videos_processed}.mp4')
            fourcc = cv2.VideoWriter_fourcc(*'mp4v')
            out = cv2.VideoWriter(output_video_path, fourcc, 30, (black_frames.shape[2], black_frames.shape[1]))

            for frame in black_frames:
                out.write(frame)

            out.release()

            videos_processed += 1

    # Print the names of skipped videos for each class
    print(f"Skipped videos in {class_folder}: {skipped_videos_per_class}")


## Preprocess data with pure Videos


In [ ]:
vid_size = (128, 128)  # Update the video size as needed
max_videos_per_class = 30  # Maximum videos to select per class
min_frames_required = 30  # Minimum frames required
videos_to_skip = 0  # Number of videos to skip in each class

def frame_crop_center(video, cropf):
    f, _, _, _ = video.shape
    startf = f // 2 - cropf // 2
    return video[startf:startf + cropf, :, :, :]

# Define the path to the main dataset folder
main_dataset_path = '/content/drive/MyDrive/vivit/Videos'  # Originial video dataset folder 


video_list = []
label_list = []
skipped_videos_per_class = {}  # To store the names of skipped videos

class_folders = os.listdir(main_dataset_path)
for class_index, class_folder in enumerate(class_folders):
    print(class_folder)
    class_folder_path = os.path.join(main_dataset_path, class_folder)

    # Track the number of videos processed for each class
    videos_processed = 0

    skipped_videos_per_class[class_folder] = []

    for filename in os.listdir(class_folder_path):
        if videos_processed >= max_videos_per_class:
            break  # Stop if we have reached the limit for this class

        if videos_processed < videos_to_skip:
            # Skip the first 'videos_to_skip' videos in each class
            videos_processed += 1
            skipped_videos_per_class[class_folder].append(filename)
            continue

        video_file_path = os.path.join(class_folder_path, filename)

        cap = cv2.VideoCapture(video_file_path)
        video_frames = []

        frame_count = 0  # Count frames in the video

        while cap.isOpened():
            ret, frame = cap.read()
            if not ret:
                break
            print(frame)
            video_frames.append(frame)
            print(type(frame))

            frame_count += 1

        cap.release()

        if frame_count >= min_frames_required:
            video = np.array(video_frames)

            # Resize and crop video frames
            L = []
            for i in range(video.shape[0]):
                frame = cv2.resize(video[i], vid_size, interpolation=cv2.INTER_CUBIC)
                L.append(frame)

            video = np.asarray(L)
            video = frame_crop_center(video, 42)

            video_list.append(video)
            label_list.append(class_index)  # Assign class index as label

            videos_processed += 1

# Convert lists to NumPy arrays
videos = np.array(video_list)
labels = np.array(label_list)

output_folder = '/content/drive/MyDrive/vivit/ProcessedData/all'
os.makedirs(output_folder, exist_ok=True)

# Save the videos and labels arrays as NumPy files
np.save('/content/drive/MyDrive/vivit/ProcessedData/all/videos.npy', videos)
np.save('/content/drive/MyDrive/vivit/ProcessedData/all/labels.npy', labels)

print("Videos shape:", videos.shape)
print("Labels shape:", labels.shape)

# Print the names of skipped videos for each class
for class_name, skipped_videos in skipped_videos_per_class.items():
    print(f"Skipped videos in {class_name}: {skipped_videos}")



